In [1]:
# Import all libraries

import numpy as np
from sklearn.cluster import DBSCAN
import pandas as pd
import matplotlib.pyplot as plt
from contextlib import redirect_stdout

In [2]:
# Interesting Atoms as input

filePath = r"C:\Users\mhanowar\OneDrive - Iowa State University\Molecular Simulation Data\convoy paper\data\dcd2-55k-65k-fixed.npy"
data = np.load(filePath)
data.shape

(10000, 6126, 3)

In [3]:
data

array([[[12.75989151,  2.25370884, 33.2609024 ],
        [12.86261272,  3.58145547, 34.02394867],
        [11.45754814,  4.32181692, 34.00331497],
        ...,
        [20.08975792, 57.91249466, 55.13398743],
        [22.04228401, 60.42287827, 57.92678833],
        [20.99219894, 58.36291122, 57.42606735]],

       [[13.43875885,  2.64563727, 33.69277573],
        [13.30926132,  3.9993763 , 34.36885834],
        [11.91111755,  4.54812336, 34.1012001 ],
        ...,
        [20.59513664, 58.44548416, 53.6669693 ],
        [22.48403168, 60.87232208, 56.73421478],
        [21.80195427, 58.67257309, 55.99963379]],

       [[12.57370472,  2.49754333, 33.2800293 ],
        [12.60299397,  3.86189485, 34.03574753],
        [11.30879116,  4.68238401, 33.93730927],
        ...,
        [21.86590576, 59.23035812, 53.45789719],
        [22.10702133, 62.1145134 , 56.61235046],
        [22.27451134, 59.72999954, 55.92777634]],

       ...,

       [[12.79883766,  2.00090075, 33.65203476],
        [12

In [4]:
# Transpose the data for Convoy algorithm

transposed_data = list()
for x in range(len(data[1])):
    
    transposed_data.append(data[:,x,0:3].tolist())

convoy_data = transposed_data


In [5]:
# View Data Shape
check_convoy_data = np.array(convoy_data)
check_convoy_data.shape

(6126, 10000, 3)

In [6]:
check_convoy_data

array([[[12.75989151,  2.25370884, 33.2609024 ],
        [13.43875885,  2.64563727, 33.69277573],
        [12.57370472,  2.49754333, 33.2800293 ],
        ...,
        [12.79883766,  2.00090075, 33.65203476],
        [13.29418087,  2.46884608, 33.80419159],
        [13.44002151,  2.66179752, 33.9376564 ]],

       [[12.86261272,  3.58145547, 34.02394867],
        [13.30926132,  3.9993763 , 34.36885834],
        [12.60299397,  3.86189485, 34.03574753],
        ...,
        [12.80395508,  3.26761055, 34.48125839],
        [13.34879589,  3.68059945, 34.67082214],
        [13.41239357,  3.87935805, 34.86313248]],

       [[11.45754814,  4.32181692, 34.00331497],
        [11.91111755,  4.54812336, 34.1012001 ],
        [11.30879116,  4.68238401, 33.93730927],
        ...,
        [11.4910841 ,  4.07874155, 34.2360611 ],
        [11.99315834,  4.35647917, 34.57761002],
        [12.08944702,  4.79071617, 34.61868668]],

       ...,

       [[20.08975792, 57.91249466, 55.13398743],
        [20

In [8]:
class ConvoyCandidate(object):
    """
    Attributes:
        indices(set): The object indices assigned to the convoy
        is_assigned (bool):
        start_time (int):  The start index of the convoy
        end_time (int):  The last index of the convoy
    """
    __slots__ = ('indices', 'is_assigned', 'start_time', 'end_time')

    def __init__(self, indices, is_assigned, start_time, end_time):
        self.indices = indices
        self.is_assigned = is_assigned
        self.start_time = start_time
        self.end_time = end_time

    def __repr__(self):
        return '<%r %r indices=%r, is_assigned=%r, start_time=%r, end_time=%r>' % (self.__class__.__name__, id(self), self.indices, self.is_assigned, self.start_time, self.end_time)

In [9]:
class CMC(object):
    """Coherence Moving Cluster (CMC) algorithm

    Attributes:
        k (int):  Min number of consecutive timestamps to be considered a convoy
        m (int):  Min number of elements to be considered a convoy
    """
    def __init__(self, clf, k, m):
        self.clf = clf
        self.k = k
        self.m = m

    def fit_predict(self, X, y=None, sample_weight=None):
        convoy_candidates = set()
        columns = len(X[0])
        column_iterator = range(columns)
        output_convoys = []

        for column in column_iterator:
            current_convoy_candidates = set()
            values = [row[column] if isinstance(row[column], (list, set)) else [row[column]] for row in X]
            if len(values) < self.m:
                continue
            clusters = self.clf.fit_predict(values, y=y, sample_weight=sample_weight)
            unique_clusters = set(clusters)
            clusters_indices = dict((cluster, ConvoyCandidate(indices=set(), is_assigned=False, start_time=None, end_time=None)) for cluster in unique_clusters)

            for index, cluster_assignment in enumerate(clusters):
                clusters_indices[cluster_assignment].indices.add(index)

            # update existing convoys
            for convoy_candidate in convoy_candidates:
                convoy_candidate_indices = convoy_candidate.indices
                convoy_candidate.is_assigned = False
                for cluster in unique_clusters:
                    cluster_indices = clusters_indices[cluster].indices
                    cluster_candidate_intersection = cluster_indices & convoy_candidate_indices
                    if len(cluster_candidate_intersection) < self.m:
                        continue
                    convoy_candidate.indices = cluster_candidate_intersection
                    current_convoy_candidates.add(convoy_candidate)
                    convoy_candidate.end_time = column
                    clusters_indices[cluster].is_assigned = convoy_candidate.is_assigned = True

                # check if candidates qualify as convoys
                candidate_life_time = (convoy_candidate.end_time - convoy_candidate.start_time) + 1
                if (not convoy_candidate.is_assigned or column == column_iterator[-1]) and candidate_life_time >= self.k:
                    output_convoys.append(convoy_candidate)

            # create new candidates
            for cluster in unique_clusters:
                cluster_data = clusters_indices[cluster]
                if cluster_data.is_assigned:
                    continue
                cluster_data.start_time = cluster_data.end_time = column
                current_convoy_candidates.add(cluster_data)
            convoy_candidates = current_convoy_candidates
        return output_convoys

In [10]:
# Clustering using DBSCAN
# distance parameter for clustering = eps

clustering_clf = DBSCAN(eps= 2.35)

# cluster = DBSCAN(eps=9, min_samples=4).fit(data[0])

# print("Number of Clusters = ", max(cluster.labels_)+1)


In [11]:
# Convoy computation

# Min elements for convoy = m
# Min consecutive timesteps = k

clf = CMC(clustering_clf, k=10, m=3)

# Convoy calculation using Test data
convoys = clf.fit_predict(convoy_data)
print("Total number of Convoys =", len(convoys))

Total number of Convoys = 10


In [12]:
# Saving the output

file = open("output.txt", "w")
file.write(f"Total number of Convoys: {len(convoys)}\n")
j = 0
for convoy in convoys:
    file.write(f"{j}. No of elements: {len(convoy.indices)}, Elements ID: {convoy.indices}, Start Time:{convoy.start_time}, End Time: {convoy.end_time}\n")
    j = j+1
#     for i in convoy.indices:
#         print(f"{i}, {data[i]}, - Start Time: {convoy.start_time}, End Time: {convoy.end_time}")

##     file.write('%r - S: %r, E: %r\n' % (convoy.indices, convoy.start_time, convoy.end_time))
file.close()